In [212]:
#Import libraries
import numpy as np
import pandas as pd


In [213]:
#Loading data
file_path = 'data.npy'
loaded_array = np.load(file_path)

In [214]:
def polar_to_cartesian(r, theta):
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return x, y


In [215]:
x_coord, y_coord = polar_to_cartesian(loaded_array[:,0], loaded_array[:,1])
scores = loaded_array[:,2]

In [216]:
def calc_gradients(x,y):
    x_non_zero = np.where(x == 0, 1, x)
    result = np.divide(y, x_non_zero)
    result = np.where(x == 0, np.nan, result)
    return result


In [217]:
slopes = calc_gradients(x_coord, y_coord)

In [218]:
intersection_y = lambda r,m,theta: r*(np.sin(theta) - m*np.cos(theta))

intersection_values_y = intersection_y(loaded_array[:,0],slopes,loaded_array[:,1])


In [219]:
intersection_x = lambda b,m: -b/m

intersection_values_x = intersection_x(intersection_values_y,slopes)

/var/folders/pn/7xb0xrx119l4_783pgg7gp140000gn/T/ipykernel_3478/1933300804.py:1: RuntimeWarning: invalid value encountered in divide
  intersection_x = lambda b,m: -b/m


In [220]:
#Getting indices of positive and negative oriented lines 

positive_gradient_lines = np.where(slopes[slopes >= 0])[0]
negative_gradient_lines = np.where(slopes[slopes < 0])[0]


In [221]:
#Create a new array that contains the x-intercept difference for each line of positive gradient to each line of negative gradient
x_intercept_diff = np.abs(intersection_values_x[positive_gradient_lines][:, np.newaxis] - intersection_values_x[negative_gradient_lines])
scores_diff = (scores[positive_gradient_lines][:, np.newaxis] + scores[negative_gradient_lines])

In [222]:

closest_pairs_data = []

for i,item in enumerate(x_intercept_diff):
    for j,element in enumerate(item):
        positive_grad = slopes[slopes >= 0][item]
        positive_intersection = intersection_values_y[item]
        negative_grad = slopes[slopes < 0][j]
        negative_intersection = intersection_values_y[j]
        x_intercept_difference = x_intercept_diff[item, j]
        sum_score = scores_diff[item, j]

        closest_pairs_data.append([
            positive_grad, negative_grad,positive_intersection,negative_intersection,
            x_intercept_difference, sum_score
        ])

column_names = ['Positive Gradient', 'Negative Gradient','Positive Y Intersection','Negative Y Intersection',
                'x-intercept Difference', 'Sum Score']

df = pd.DataFrame(closest_pairs_data, columns=column_names)

df

IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
#df.to_csv('closest_pairs.csv', index=False)
